## はじめに

このページは、特定のアプリケーション向けのGPT Actionを構築する開発者向けの手順とガイドを提供します。進める前に、まず以下の情報に慣れ親しんでおくことを確認してください：
- [GPT Actionsの紹介](https://platform.openai.com/docs/actions/introduction)
- [GPT Actionsライブラリの紹介](https://platform.openai.com/docs/actions/actions-library)
- [GPT Actionをゼロから構築する例](https://platform.openai.com/docs/actions/getting-started)

この特定のGPT Actionは、Salesforce、特にSalesforce Service Cloudへの接続方法の概要を提供します。このActionで詳述されているスキーマにより、ユーザーはChatGPTから直接ケースデータを取得し、ケースを更新することができます。他のSalesforce Cloudソリューション用のActionsを作成するセットアッププロセスでは、同じConnected Appと認証設定を使用しますが、異なるAPIスキーマが必要になります。

### 価値 + ビジネス活用事例

**価値**: ユーザーはChatGPTの自然言語機能を活用して、Salesforceに直接接続できるようになりました

**使用例**: 
- 顧客への平均応答時間の短縮
- ケースや問題のトラブルシューティング時間の短縮
- GPTの知識と指示と組み合わせることで、顧客対応における一貫したブランドボイスの確保

## アプリケーション情報

### アプリケーションキーリンク

開始する前に、アプリケーションから以下のリンクを確認してください：
- [SalesforceでLightningアプリを作成する](https://help.salesforce.com/s/articleView?id=sf.apps_lightning_create.htm&type=5)
- [OAuthトークンとスコープ](https://help.salesforce.com/s/articleView?id=sf.remoteaccess_oauth_tokens_scopes.htm&type=5)
- [Salesforce APIドキュメント](https://developer.salesforce.com/docs/apis)

### アプリケーションの前提条件

開始する前に、アプリケーション環境で以下の手順を実行していることを確認してください：
- SalesforceでAppを作成する権限があることを確認する

## ChatGPT ステップ

### カスタムGPT指示

Custom GPTを作成したら、以下のテキストをInstructionsパネルにコピーしてください。質問がありますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をご確認ください。

In [ ]:
**Context**: Your purpose is to pull information from Service Cloud, and push updates to cases. A user is going to ask you a question and ask you to make updates.

**Instructions**:
1. When a user asks you to help them solve a case in Service Cloud, ask for the case number and pull the details for the case into the conversation using the getCaseDetailsFromNumber action.
2. If the user asks you to update the case details, use the action updateCaseStatus.

**Example**: 
User: Help me solve case 00001104 in Service Cloud.

### OpenAPI スキーマ

Custom GPTを作成したら、以下のテキストをActionsパネルにコピーしてください。ご質問がある場合は、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をチェックして、この手順がどのように動作するかの詳細をご確認ください。

In [ ]:
openapi: 3.1.0
info:
  title: Salesforce Service Cloud Case Update API
  description: API for updating the status of Service Cloud tickets (cases) in Salesforce.
  version: 1.0.3
servers:
  - url: https://your_instance.my.salesforce.com
    description: Base URL for your Salesforce instance (replace 'your_instance' with your actual Salesforce domain)
paths:
  /services/data/v60.0/sobjects/Case/{CaseId}:
    patch:
      operationId: updateCaseStatus
      summary: Updates the status of a Service Cloud case
      description: Updates the status of a Service Cloud ticket based on the case ID number.
      parameters:
        - name: CaseId
          in: path
          required: true
          description: The ID of the case to update.
          schema:
            type: string
      requestBody:
        required: true
        content:
          application/json:
            schema:
              type: object
              properties:
                Status:
                  type: string
                  description: The new status of the case.
      responses:
        '204':
          description: Successfully updated the case status
        '400':
          description: Bad request - invalid input or case ID not found
        '401':
          description: Unauthorized - authentication required
        '404':
          description: Not Found - case ID does not exist
    delete:
      operationId: deleteCase
      summary: Deletes a Service Cloud case
      description: Deletes a Service Cloud ticket based on the case ID number.
      parameters:
        - name: CaseId
          in: path
          required: true
          description: The ID of the case to delete.
          schema:
            type: string
      responses:
        '204':
          description: Successfully deleted the case
        '400':
          description: Bad request - invalid case ID
        '401':
          description: Unauthorized - authentication required
        '404':
          description: Not Found - case ID does not exist
  /services/data/v60.0/query:
    get:
      operationId: getCaseDetailsFromNumber
      summary: Retrieves case details using a case number
      description: Retrieves the details of a Service Cloud case associated with a given case number.
      parameters:
        - name: q
          in: query
          required: true
          description: SOQL query string to find the Case details based on Case Number.
          schema:
            type: string
            example: "SELECT Id, CaseNumber, Status, Subject, Description FROM Case WHERE CaseNumber = '123456'"
      responses:
        '200':
          description: Successfully retrieved the case details
          content:
            application/json:
              schema:
                type: object
                properties:
                  totalSize:
                    type: integer
                  done:
                    type: boolean
                  records:
                    type: array
                    items:
                      type: object
                      properties:
                        Id:
                          type: string
                        CaseNumber:
                          type: string
                        Status:
                          type: string
                        Subject:
                          type: string
                        Description:
                          type: string
        '400':
          description: Bad request - invalid query
        '401':
          description: Unauthorized - authentication required
        '404':
          description: Not Found - case number does not exist


## 認証手順

以下は、このサードパーティアプリケーションで認証を設定する手順です。ご質問がございますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をご確認ください。

### 事前アクション手順

ChatGPTで認証を設定する前に、アプリケーションで以下の手順を実行してください。

ChatGPTで認証を設定する前に、アプリケーションで以下の手順を実行してください。
1. Salesforce Setupに移動する

![gptactions_salesforce.png](../../../images/gpt_actions_salesforce_setup.png)

2. "App Manager"を検索する

![gptactions_salesforce.png](../../../images//gpt_actions_salesforce_manager.png)

3. 「New Connected App」をクリック
4. Connected App Nameを入力
5. 連絡先メールアドレス（あなたのメールアドレス）を入力
6. OAuth設定を有効にするチェックボックスにチェックを入れる
7. コールバックURLを挿入（今のところhttps://chat.openai.com/aip//oauth/callbackのようなプレースホルダーを使用し、後でChatGPTでActionを作成する際に更新します）

![gptactions_salesforce.png](../../../images//gpt_actions_salesforce_oauth2.png)

8. 「Selected OAuth Scopes」を選択し、適切な権限を付与します。これらのスコープは、社内のセキュリティポリシーに基づいて設定してください。

![gptactions_salesforce.png](../../../images//gpt_actions_salesforce_scope.png)

9. 以下のボックスにチェックが入っていることを確認してください：
- Enable Client Credentials Flow
- Enable Authorization Code and Credentials FLow
- Enable Token Exchange Flow
10. 以下のボックスのチェックが外れていることを確認してください：
- Require Proof Key for Code Exchange (PKCE) Extension for Supported Authorization Flows

![gptactions_salesforce.png](../../../images/gpt_actions_salesforce_settings_condensed.png)

11. 新しいConnected Appを保存する
12. 「Consumer Key and Secret」の下にある「Manage Consumer Details」をクリックします。アカウントにメールで送信されたコードを使用してアクセスを確認し、キーとシークレットをコピーします。
- Salesforce Consumer Key = ChatGPT Client ID
- Salesforce Consumer Secret = ChatGPT Client Secret

![gptactions_salesforce.png](../../../images//gpt_actions_salesforce_credentials.png)

13. アプリページに戻る
14. 「Manage」をクリック
15. 「Edit Policies」をクリック
16. OAuth Policiesの下で、「Enable Token Exchange Flow」ボックスにチェックを入れる

![gptactions_salesforce.png](../../../images//gpt_actions_salesforce_token.png)

17. 保存をクリックしてください！

### ChatGPTにおいて

ChatGPTで、「Authentication」をクリックし、**「OAuth」**を選択してください。以下の情報を入力してください。

- **Client ID**: 上記の手順で取得したClient IDを使用
- **Client Secret**: 上記の手順で取得したClient Secretを使用
- **Authorization URL**: https://[inserturlhere].my.salesforce.com/services/oauth2/authorize
- **Token URL**: https://[inserturlhere].my.salesforce.com/services/oauth2/token 
- **Scope**: full
- **Token**: Default (POST)

### アクション後のステップ

ChatGPTで認証を設定したら、アプリケーション内で以下の手順に従ってActionを完了してください。

- GPT ActionからコールバックURLをコピーする
- Salesforceの接続アプリに戻り、コールバックURLを追加する

### FAQ & トラブルシューティング

- *コールバックURLエラー:* ChatGPTでコールバックURLエラーが発生した場合は、上記のスクリーンショットに注意深く従ってください。アクションが正しく認証されるように、コールバックURLをSalesforceに直接追加する必要があります
- *内部サーバーエラー:* 接続アプリのOAuth設定で、すべての正しいボックスがチェックされている、または未チェックになっていることを確認してください。

*優先して対応してほしい統合機能はありますか？統合機能にエラーがありますか？GitHubでPRやissueを作成していただければ、確認いたします。*